* Find file location on disk
  * Place under Git
  * Create new IHE-SDC Git repo for SDC Object Model Usage Samples
  * Create a folder for SDC XML files
  * Create "Assmblies" folder for imported assemblies
    * Copy in the latest SDC.Schema dll 
    * Copy in "C:\Program Files\Microsoft Visual Studio\2022\Enterprise\Common7\IDE\PublicAssemblies\Microsoft.VisualStudio.QualityTools.UnitTestFramework.dll" //Microsoft.VisualStudio.TestTools.UnitTesting
    * Create Archive folder for junk notebooks

In [ ]:
//Import assemblies from files
#r "./Assemblies/SDC_CodeGeneratorTest.dll"  //SDC.Schema namespace
#r "./Assemblies/Microsoft.VisualStudio.QualityTools.UnitTestFramework.dll"  //unit test framework, if we want to use Asserts etc.

using System;
using SDC.Schema;
//
    //using SDC;
    //using System.Runtime;
    //using System.IO;

    using Microsoft.VisualStudio.TestTools.UnitTesting;
    //using Newtonsoft.Json;
    //using System.Data;
    //using System.Diagnostics;
    //using System.Drawing;
    //using System.Linq;
    //using System.Xml; 

* Read in a large SDC file, and then deserialize as XML

In [ ]:
string path = @".\SDC XML\Breast.Invasive.Staging.359_.CTP9_sdcFDF.xml";
var FD = SDC.Schema.FormDesignType.DeserializeFromXmlPath(path); //Deserialize
string myXML =  SdcSerializer<FormDesignType>.Serialize(FD); //Serialize
//myXML

## Create iterator to return nodes

In [ ]:
using System.Diagnostics;

////SDC Setup
    BaseType.ResetSdcImport();
    string path = @".\SDC XML\Breast.Invasive.Staging.359_.CTP9_sdcFDF.xml";
    var FD = SDC.Schema.FormDesignType.DeserializeFromXmlPath(path); //Deserialize

////TIMER SETUP
    Stopwatch.StartNew();
    var timerStartTime  = (float)Stopwatch.GetTimestamp();
    display(timerStartTime);

int i = 0;

foreach (BaseType node in NodeIterator(FD))
    {
        //btPrint(node);
    }
///////////////////////////////////////////////////
IEnumerable<BaseType> NodeIterator(BaseType? n)
    {
        while (n is not null)
        {
            n = MoveNext(n);
            if (n is not null)
                yield return n;
            else yield break;
        }
        yield break;
    }
/////////////////////////////////////////////////
BaseType? MoveNext(BaseType n)
{
    Dictionary<Guid, List<BaseType>> cn = n.TopNode.ChildNodes;
    List<BaseType>? childList;
    BaseType? nextNode;
    
    n.order = i;  //almost instananeous
    Assert.IsTrue(n.ObjectID == i);//very fast
    i++;
    //if n has child nodes, the next node is the first child node of n.
    if (cn.TryGetValue(n.ObjectGUID, out childList))
    {
        nextNode = childList[0];
        if (nextNode is not null) return nextNode;
    }
    //Notes:
        //n has no child nodes here, so walk up the tree to the parent node.
        //When we walk back up the object graph, prevPar is the original starting node (deeper in the tree),
        //while par is the parent of prevPar.  par is more superficial in the tree, and closer to the top node
        //We then check the childList of par, to see if prePar can be found in that childList
        //If prevPar is in childList, then try to retrieve the next node in childList
        //IF we cant get the nextNode from childList, then move up to one higher parent level
    var prevPar = n;  
    var par = prevPar.ParentNode;

    while (par is not null)
    {
        if (cn.TryGetValue(par.ObjectGUID, out childList))
        {
            var index = childList.IndexOf(prevPar);
            if (index < childList.Count - 1)
            {
                nextNode = childList[index + 1];
                if (nextNode is not null) return nextNode;
            }
            //the next node is not located yet, so walk up to a previous ancestor and try again,
                //looking in that ancestors childList for a nextNode candidate
            prevPar = par; 
            par = prevPar.ParentNode;
        }
    }
    return null;
}
/////////////////////////Small method to output node info///////////////////////////////
void btPrint(BaseType n)
    {
        string content;
        if (n is not null)
        {
            if (n is DisplayedType) content = ": title: " + (n as DisplayedType)?.title;
            else if (n is PropertyType) content = ": " + (n as PropertyType)?.propName + ": " + (n as PropertyType)?.val;
            else content = "";

            display(n.ObjectID.ToString().PadLeft(4) + ": " + i.ToString().PadLeft(4) + ": " + (n.name ?? "").PadRight(20) + ": " + (n.ElementName ?? "").PadRight(25) + content);
        }
    }
display(Stopwatch.GetTimestamp());
display(((Stopwatch.GetTimestamp() - timerStartTime) / Stopwatch.Frequency).ToString());

2.6070124E+12

2607012410938

0

## Find SDC node changes compared to the reference template
* Create 2 FormDesign OMs, "newOM" and "refOM"
* Iterate each node in newOM, and locate matcching node in refOM by ID, SGuid and/or name
  * For each newOM node, determine if it's 
    * new, 
    * has a different parent node, 
    * previous sib has changed.
  * Add the changed node's ID/name/sGUID and change type to a dictionary
* Compare each SDC property in newOM with the matching refOM node
* 

In [ ]:
// var a1 = new { A = 1, B = 2, C = 3, D = 4, E = 5 };
// var a2 = a1 with { E = 10 }; 
// Console.WriteLine (a2);      // { A = 1, B = 2, C = 3, D = 4, E = 10 }

public readonly record struct Q(
    string ID, string name, string type, string styleClass, //BaseType
    string title, bool enabled, bool visible, bool mustImplement, bool showInReport,
    uint minCard, uint maxCard,
    uint repeat, string instanceGUID, string parentGUID,
    bool readOnly, bool changedData, bool newData
);

public readonly record struct btDiff (BaseType btOld , string ChangedPropName , string ChangedPropVal);
var Xchanges = new Dictionary<string, btDiff> ();

bool BaseTypeCompare(BaseType btOld, BaseType btNew, Dictionary<string, btDiff> btChanges)
    {
        bool changed = false;
        
        if (btOld.name != btNew.name) {changed = true; btChanges.Add(btOld.name, new btDiff(btOld, nameof(btOld.name), btOld.name));}
        if (btOld.type != btNew.type) {changed = true; btChanges.Add(btOld.name, new btDiff(btOld, nameof(btOld.type), btOld.type));}
        if (btOld.styleClass != btNew.styleClass) {changed = true; btChanges.Add(btOld.name, new btDiff(btOld, nameof(btOld.styleClass), btOld.styleClass));}
        if (btOld.sGuid != btNew.sGuid) {changed = true; btChanges.Add(btOld.name, new btDiff(btOld, nameof(btOld.sGuid), btOld.sGuid));}
        if (btOld.order != btNew.order) {changed = true; btChanges.Add(btOld.name, new btDiff(btOld, nameof(btOld.order), btOld.order.ToString()));}

        if (btOld.ParentNode.name != btNew.ParentNode.name) {changed = true; btChanges.Add(btOld.name, new btDiff(btOld, "ParentNode.name", btOld.ParentNode.name));}
        if (btOld.ParentIETypeNode.name != btNew.ParentIETypeNode.name) {changed = true; btChanges.Add(btOld.name, new btDiff(btOld, "ParentIETypeNode.name", btOld.ParentIETypeNode.name));}
        if (btOld.ParentIETypeID != btNew.ParentNode.ParentIETypeID) {changed = true; btChanges.Add(btOld.name, new btDiff(btOld, nameof(btOld.ParentIETypeID), btOld.ParentIETypeID));}
        
        //Check previous sibling match, if present
        var prevSibOld = btOld.GetNodePreviousSib();
        var prevSibNew = btOld.GetNodePreviousSib();
        if(prevSibOld.name != prevSibNew.name)
        {changed = true; btChanges.Add(btOld.name, new btDiff(btOld, "PreviousSib.name", prevSibOld?.name??""));}

        //Check node type
        if (btOld.GetType().Name != btNew.GetType().Name) {changed = true; btChanges.Add(btOld.name, new btDiff(btOld, "GetType.Name", btOld.GetType().Name));}
        //Check parent node type
        if (btOld.ParentNode.GetType().Name != btNew.ParentNode.GetType().Name) {changed = true; btChanges.Add(btOld.name, new btDiff(btOld, "ParentNode.GetType.Name", btOld.ParentNode.ParentNode.GetType().Name));}
        //Check all Properties, Comments, Extensions here?

        if (changed) return false;            

        return true;
    }
    bool QuestionCompare(QuestionItemType btOld, QuestionItemType btNew, Dictionary<string, btDiff> btChanges)
    {
        bool changed = BaseTypeCompare(btOld, btNew, btChanges);
        //IET
        if (btOld.ID != btNew.ID) {changed = true; btChanges.Add(btOld.name, new btDiff(btOld, nameof(btOld.ID), btOld.ID));}
        if (btOld.baseURI != btNew.baseURI) {changed = true; btChanges.Add(btOld.name, new btDiff(btOld, nameof(btOld.baseURI), btOld.baseURI));}

        //DisplayedType
        if (btOld.title != btNew.title) {changed = true; btChanges.Add(btOld.name, new btDiff(btOld, nameof(btOld.title), btOld.title));}
        if (btOld.enabled != btNew.enabled) {changed = true; btChanges.Add(btOld.name, new btDiff(btOld, nameof(btOld.enabled), btOld.enabled.ToString()));}
        if (btOld.visible != btNew.visible) {changed = true; btChanges.Add(btOld.name, new btDiff(btOld, nameof(btOld.visible), btOld.visible.ToString()));}
        if (btOld.mustImplement != btNew.mustImplement) {changed = true; btChanges.Add(btOld.name, new btDiff(btOld, nameof(btOld.mustImplement), btOld.mustImplement.ToString()));}
        if (btOld.showInReport != btNew.showInReport) {changed = true; btChanges.Add(btOld.name, new btDiff(btOld, nameof(btOld.showInReport), btOld.showInReport.ToString()));}

        //RepeatType
        if (btOld.minCard != btNew.minCard) {changed = true; btChanges.Add(btOld.name, new btDiff(btOld, nameof(btOld.minCard), btOld.minCard.ToString()));}
        if (btOld.maxCard != btNew.maxCard) {changed = true; btChanges.Add(btOld.name, new btDiff(btOld, nameof(btOld.maxCard), btOld.maxCard.ToString()));}
        if (btOld.repeat != btNew.repeat) {changed = true; btChanges.Add(btOld.name, new btDiff(btOld, nameof(btOld.repeat), btOld.repeat.ToString()));}
        if (btOld.instanceGUID != btNew.instanceGUID) {changed = true; btChanges.Add(btOld.name, new btDiff(btOld, nameof(btOld.instanceGUID), btOld.instanceGUID));}
        if (btOld.parentGUID != btNew.parentGUID) {changed = true; btChanges.Add(btOld.name, new btDiff(btOld, nameof(btOld.parentGUID), btOld.parentGUID));}

        //QuestionItemBaseType
        if (btOld.readOnly != btNew.readOnly) {changed = true; btChanges.Add(btOld.name, new btDiff(btOld, nameof(btOld.readOnly), btOld.readOnly.ToString()));}
        if (btOld.changedData != btNew.changedData) {changed = true; btChanges.Add(btOld.name, new btDiff(btOld, nameof(btOld.changedData), btOld.changedData.ToString()));}
        if (btOld.newData != btNew.newData) {changed = true; btChanges.Add(btOld.name, new btDiff(btOld, nameof(btOld.newData), btOld.newData.ToString()));}
        if (btOld.GetQuestionSubtype() != btNew.GetQuestionSubtype()) {changed = true; btChanges.Add(btOld.name, new btDiff(btOld, "GetQuestionSubtype"), btOld.GetQuestionSubtype().ToString()));}
        
        if (btOld.GetQuestionSubtype() == QuestionEnum.QuestionFill)
        {
            //Look for some LIR properties, like datatype, minInclusive, maxInclusive, fractionDigits, val, quantEnum, etc.
            //Also TextAfterResponse, Response Units
        }
        if (btOld.GetQuestionSubtype() == QuestionEnum.QuestionSingleOrMultiple)
        {
            //minSelections, maxSelections, numCols
        }

        if (changed) return false;            

        return true;
    }


In [ ]:
var bt = new FormDesignType();

if(null == null) display(true);

True